In [30]:
import xml.etree.ElementTree as ET
from PIL import Image
import os
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm

Найдем все файлы с расширением ".json"

In [31]:
anno_pathes_json = []
file_name = '.json'

absolute_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\TV_data'
crop_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops'

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

for root, dirs, files in os.walk(absolute_path):    
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (file_name in file) & ('.jsonl' not in file):
            # если есть, то добавляем путь в список
            anno_pathes_json.append(os.path.join(root, file)) 

Сделаем кроп изображений содержащих цифры из поолученной разметки

In [32]:
alpha = 0.1
num = 0
annos_list = []
anno_dict = {'file_name': [], 'text': []}
pathes_list = []

for anno in tqdm(anno_pathes_json):
    with open(anno, 'r') as f:
        data = json.load(f)

    #Преобразуем json  в dataFrame
    points = pd.read_json(json.dumps(data['shapes']))
    #file_name = pd.read_json(json.dumps(data["imagePath"]))
    x = pd.read_json(anno, orient='index')   
    frame = x.iloc[3, :][0]
    frame = frame.replace('.', '-')
 
    img_path = anno.replace('anno', 'frames')
    img_path = img_path.replace('.json', '.jpeg')
    img_path = img_path.replace("BallerTV", "TV_data")
    img = Image.open(img_path)

    for j in range(len(points)):
        out_file = {}
        point_obj = pd.DataFrame(points.iloc[j, 0:2])
        point = point_obj.iloc[1, :]
        label = point_obj.iloc[0, :]
        split_label = label[j].split('_')
        number = split_label[-1]

        for box in point:   

            if number.isdigit() & (len(split_label) == 3):

                xtl, ytl, xbr, ybr = int(box[0][0]), int(box[0][1]), int(box[1][0]), int(box[1][1])
                            
                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                # Сделаем кроп номера на футболке
                croppped_image = img.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(number)                

                if not os.path.exists(full_path):
                    os.makedirs(full_path)    
                    #print(f'Папка {number} создана')  
                                  
                res_path = full_path +'\\' + f'TV_data_{str(frame)}_example.jpg'
                num += 1

                annos_list.append(str(number))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list
                
                croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_first.csv")
df.head()

  0%|          | 0/5598 [00:00<?, ?it/s]

,file_name,text
0,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,33
1,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,30
2,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,33
3,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,30
4,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,33


Найдем все пути к файлам "annotations.xml"

In [33]:
crop_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops"
absolute_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\TV_data"

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

file_name = 'annotations.xml'
jpeg_name = '.jpeg'

anno_pathses = {}
img_pathses = {}
# пройдемся рекурсивно по всем папкам, начиная с absolute_path
for root, dirs, files in os.walk(absolute_path):    

    # проверяем, есть ли заданный файл в списке файлов текущей папки
    folder_name = os.path.split(root)
    #(folder_name[1] in root) & ('frames' in root)  & 

    if file_name in files:
        # если есть, то добавляем путь в список
        anno_pathses[folder_name[1]] = os.path.join(root, file_name)

    for file in files:
        
        if (jpeg_name in file) & (folder_name[1] not in img_pathses) & ('0_processed' not in root) & ('00-00-00' not in root):
            img_pathses[folder_name[1]] = root    

Сделаем кроп изображений содержащих цифры из поолученной разметки

In [34]:
nums = []
num = 0
folder_pathes = []
pathes_list = []
annos_list = []
anno_dict = {'file_name': [], 'text': []}
num = 0
alpha = 0.2
res = {}

for key in tqdm(anno_pathses.keys()):

    # Загрузим XML файл
    tree = ET.parse(anno_pathses[key])

    # Получим корневой элемент XML файла
    root = tree.getroot()

    # Проитерируемся по дочерним элементам корня
    for image in root.findall('image'):
        
        image_id = image.get('id')
        file_name = image.get('name')
        file_name = os.path.split(file_name)
        folder_name = os.path.split(file_name[0])
        file_name = os.path.join(img_pathses[key], file_name[1])
        if key not in file_name:
            file_name = file_name.replace(key, folder_name)
    
        # Переберем все baundingbox`s
        for box in image.findall('box'):
            
            image = Image.open(file_name)
            
            label = box.get('label')

            # Проверим является ли метка меткой номера
            if label == "Jersey":
                xtl = int(float(box.get('xtl')))
                ytl = int(float(box.get('ytl')))
                xbr = int(float(box.get('xbr')))
                ybr = int(float(box.get('ybr')))
                number = box.find('attribute[@name="number"]').text
                if number == '-1':
                    continue
                
                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                # Сделаем кроп номера на футболке
                croppped_image = image.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(number)
                num += 1

                if not os.path.exists(full_path):
                    os.makedirs(full_path)    
                    #print(f'Папка {number} создана')  

                res_path = full_path +'\\' + f'TV_data_{str(key)}_id-{image_id}.jpg'

                annos_list.append(int(number))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list
                
                croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_second.csv")
df.head()


  0%|          | 0/104 [00:00<?, ?it/s]

,file_name,text
0,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,1
1,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,8
2,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,11
3,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,3
4,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,3


Найдем все файлы с расширением ".json"

In [35]:
anno_pathes_json = []
file_name = '.json'

absolute_path = 'data\\basketball\\16-11-2022'
crop_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops'

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

for root, dirs, files in os.walk(absolute_path):    
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (file_name in file) & ('.jsonl' not in file):
            # если есть, то добавляем путь в список
            anno_pathes_json.append(os.path.join(root, file)) 

Сделаем кроп изображений содержащих цифры из поолученной разметки

In [36]:
num = 0
alpha = 0.2
annos_list = []
anno_dict = {'file_name': [], 'text': []}
pathes_list = []
xx = []

for i, anno in enumerate(anno_pathes_json):
    with open(anno, 'r') as f:
        data = json.load(f)
       

    frames = pd.read_json(json.dumps(data['frames']))
    print(f'{i + 1}ая папка из {len(anno_pathes_json)}')


    for i in tqdm(range(len(frames))):

        img_name = frames['img_name'][i]
        bboxes = frames['detections'][i]
        #print(bboxes)

        anno_path = anno.replace('anno', 'frames')
        anno_path = anno_path.replace('.json', '\\')
        anno_path_fin = anno_path + str(img_name)
        xx.append(anno_path)

        #image = cv2.cvtColor(cv2.imread(anno_path_fin), cv2.COLOR_BGR2RGB)
        img = Image.open(anno_path_fin)

        # Создание объекта ImageDraw для рисования
        #draw = ImageDraw.Draw(image)

        for detection in bboxes:
        
            ocr_jersey = detection['ocr_jersey']
            '''
            if ocr_jersey == '-1':
                print(xx[-1], f'Label {ocr_jersey}')
                break
            '''    
            if ocr_jersey != '-1':
                jersey_bbox_global = detection['jersey_bbox_global']
                xtl = int(jersey_bbox_global[0])
                ytl = int(jersey_bbox_global[1])
                xbr = int(jersey_bbox_global[2])           
                ybr = int(jersey_bbox_global[3])
                
                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                #draw.rectangle([(jersey_bbox_global[0], jersey_bbox_global[1]), (jersey_bbox_global[2], jersey_bbox_global[3])], outline='red')
                #cv2.rectangle(image, (xtl, ytl), (xbr, ybr), 2)
                croppped_image = img.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(ocr_jersey)
                res_path = full_path + '\\' f'basketball_{num}example.jpg'
                num += 1

                if not os.path.exists(full_path):
                        os.makedirs(full_path)    

                annos_list.append(str(ocr_jersey))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list

                croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_basketball.csv")
df.head()

1ая папка из 22


  0%|          | 0/500 [00:00<?, ?it/s]

2ая папка из 22


  0%|          | 0/500 [00:00<?, ?it/s]

3ая папка из 22


...

21ая папка из 22


  0%|          | 0/500 [00:00<?, ?it/s]

22ая папка из 22


  0%|          | 0/500 [00:00<?, ?it/s]

,file_name,text
0,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,1
1,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,3
2,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,1
3,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,3
4,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,1


Найдем все пути к файлам с именем "annotations.xml"

In [37]:
crop_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops"
absolute_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\FIBA3x3\\fiba3x3_tracking_135"

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

file_name = 'annotations.xml'

anno_pathses = []
# пройдемся рекурсивно по всем папкам, начиная с absolute_path
for root, dirs, files in os.walk(absolute_path):    
    # проверяем, есть ли заданный файл в списке файлов текущей папки
    if file_name in files:
        # если есть, то добавляем путь в список
        anno_pathses.append(os.path.join(root, file_name))  

len(anno_pathses)

135

Сделаем кроп изображений содержащих цифры из поолученной разметки

In [38]:
nums = []
num = 0
folder_pathes = []
pathes_list = []
annos_list = []
anno_dict = {'file_name': [], 'text': []}

alpha = 0.2

for path in tqdm(anno_pathses):

    # Загрузим XML файл
    tree = ET.parse(path)

    # Получим корневой элемент XML файла
    root = tree.getroot()

    # Проитерируемся по дочерним элементам корня
    for image in root.findall('image'):
        
        image_id = image.get('id')
        file_name = image.get('name')
        #file_name = file_name.replace('/', '\\')
        file_name = os.path.join(absolute_path, file_name)
        if 'fiba3x3/images/331/' in file_name:
            file_name = file_name.replace('fiba3x3/images/331', 'frames')
        file_name = file_name.replace('frames', 'frames_135')
        # Переберем все baundingbox`s
        for box in image.findall('box'):
            
            image = Image.open(file_name)
           
            label = box.get('label')

            

            # Проверим является ли метка меткой номера
            if label == "Jersey":
                xtl = int(float(box.get('xtl')))
                ytl = int(float(box.get('ytl')))
                xbr = int(float(box.get('xbr')))
                ybr = int(float(box.get('ybr')))
                number = box.find('attribute[@name="number"]').text

                if str(number) == '-1':
                    continue

                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                # Сделаем кроп номера на футболке
                croppped_image = image.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(number)
                num += 1

                if not os.path.exists(full_path):
                    os.makedirs(full_path)    
                    #print(f'Папка {number} создана')  
                    
                res_path = full_path +'\\' + f'fiba_{str(num)}example.jpg'

                annos_list.append(int(number))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list
                
                croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_fiba.csv")
df.head()

  0%|          | 0/135 [00:00<?, ?it/s]

,file_name,text
0,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,4
1,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,7
2,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,4
3,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,4
4,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,4


Найдем все файлы с расширением ".json"

# Доработать!!!


In [7]:
anno_pathes_json = []
img_pathes = {}
file_name = '.json'
img_name = '.jpg'

absolute_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\NCAA\\NCAA_1_jersey\\second_task'
crop_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops'
img_folder_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\NCAA\\NCAA_1_tracking\\playerTrackingFrames'

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

for root, dirs, files in os.walk(absolute_path):      
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (file_name in file) & ('.jsonl' not in file):
            # если есть, то добавляем путь в список
            anno_pathes_json.append(os.path.join(root, file)) 

for root, dirs, files in os.walk(img_folder_path):        
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (img_name in file):
            # если есть, то добавляем путь в список
            sh_file = file.replace('frame_', '')
            img_pathes[sh_file] = os.path.join(root, file)  

Сделаем кроп изображений содержащих цифры из поолученной разметки

In [29]:
alpha = 0.1
num = 0
annos_list = []
anno_dict = {'file_name': [], 'text': []}
pathes_list = []

for anno in tqdm(anno_pathes_json):
    with open(anno, 'r') as f:
        data = json.load(f)

    #Преобразуем json  в dataFrame
    points = pd.read_json(json.dumps(data['shapes']))
    img_name = str(json.dumps(data['imagePath']))
    img_path = img_name.replace('"', '')
    img_path = os.path.split(img_path)
    img_path = img_pathes[img_path[-1]]

    img = Image.open(img_path)
        
    for i, shape in points.iterrows():
        out_file = {}    

        if shape["shape_type"] == "rectangle":                       
            
            label = shape['label']       
            split_label = label.split('_')
            number = str(split_label[-1])

            if len(split_label) == 3:            
                
                point_obj = shape['points']
                singl_point = point_obj
                [xtl, ytl], [xbr, ybr] = singl_point

                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                # Сделаем кроп номера на футболке
                croppped_image = img.crop((xtl, ytl, xbr, ybr))
                if str(number) == '5(1)':
                    number = '5' 
                full_path = crop_path + '\\' + str(number)                

                if not os.path.exists(full_path):
                    os.makedirs(full_path)  
                    #print(label)  
                    #print(f'Папка {number} создана')                    
                                    
                res_path = full_path +'\\' + f'ncaa_{str(num)}example.jpg'
                num += 1

                annos_list.append(str(number))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list
                    
                croppped_image.save(res_path)
  
df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_ncaa.csv")
df.head()  

  0%|          | 0/6965 [00:00<?, ?it/s]

KeyboardInterrupt: 